In [1]:
import os
os.chdir("/home/am23mtech11006/AM23MTECH11006/Physics informed_laser")

import sys
sys.path.append('./src')
import numpy as np
import torch
torch.manual_seed(0)
import torch.nn as nn
from torch.autograd import Variable,grad
from src.model import FNN
from src.util import *
from src.train import *

In [2]:
def PDE(x,y,z,t,net):
    X = torch.cat([x,y,z,t],axis=-1)
    T = net(X)
    
    T_t = grad(T,t,create_graph=True,grad_outputs=torch.ones_like(T))[0]

    T_x = grad(T,x,create_graph=True,grad_outputs=torch.ones_like(T))[0]
    T_xx = grad(T_x,x,create_graph=True,grad_outputs=torch.ones_like(T_x))[0]
    
    T_y = grad(T,y,create_graph=True,grad_outputs=torch.ones_like(T))[0]
    T_yy = grad(T_y,y,create_graph=True,grad_outputs=torch.ones_like(T_y))[0]
    
    T_z = grad(T,z,create_graph=True,grad_outputs=torch.ones_like(T))[0]
    T_zz = grad(T_z,z,create_graph=True,grad_outputs=torch.ones_like(T_z))[0]
    
    f = rho*Cp*T_t - k*(T_xx+T_yy+T_zz)

    return f

In [3]:
def generate_points(p=[],f=[]):

    t = np.linspace(x_min[3]+0.01,x_max[3],61)

    # boundary points
    bound_x_neg,_ = sampling_uniform(1.,x_min,x_max,'-x',t)
    bound_x_pos,_ = sampling_uniform(1.,x_min,x_max,'+x',t)

    bound_y_neg,_ = sampling_uniform(1.,x_min,x_max,'-y',t)
    bound_y_pos,_ = sampling_uniform(1.,x_min,x_max,'+y',t)

    bound_z_neg,_ = sampling_uniform(1.,x_min,x_max,'-z',t)
    bound_z_pos,_ = sampling_uniform(1.,x_min,x_max,'+z',t)

    bound_z_pos_more = [] # more points for surface flux
    
    for ti in t:
        if ti<=t_end:
            zi,_ = sampling_uniform(.25,
                        [max(x0+ti*v-2*r,x_min[0]),max(x_min[1],y0-2*r),x_min[2]],
                        [min(x0+ti*v+2*r,x_max[0]),min(x_max[1],y0+2*r),x_max[2]],
                        '+z',[ti])
            bound_z_pos_more.append(zi)

    bound_z_pos_more = np.vstack(bound_z_pos_more)
    bound_z_pos = np.vstack((bound_z_pos,bound_z_pos_more))

    ### domain points
    domain_pts1,_ = sampling_uniform(2.,
                                     [x_min[0],x_min[1],x_min[2]],
                                     [x_max[0],x_max[1],x_max[2]-3.],'domain',t)

    domain_pts2,_ = sampling_uniform(1.,
                                     [x_min[0],x_min[1],x_max[2]-3.+.5],
                                     [x_max[0],x_max[1],x_max[2]-1.],'domain',t)

    domain_pts3 = []
    for ti in t:
        di,_ = sampling_uniform(.5,
                                [x_min[0],x_min[1],x_max[2]-1.+.25,],
                                [x_max[0],x_max[1],x_max[2]],'domain',[ti])
        domain_pts3.append(di)
    domain_pts3 = np.vstack(domain_pts3)
    domain_pts = np.vstack((domain_pts1,domain_pts2,domain_pts3))

    # initial points
    init_pts1,_ = sampling_uniform(2.,[x_min[0],x_min[1],x_min[2]],
                                   [x_max[0],x_max[1],x_max[2]],'domain',[0],e=0)
    # more points near the toolpath origin
    init_pts2,_ = sampling_uniform(.5,[x0-2,y0-2,x_max[2]-2],
                                   [x0+2,y0+2,x_max[2]],'domain',[0])
    
    init_pts = np.vstack((init_pts1,init_pts2))
    

    p.extend([torch.tensor(bound_x_neg,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_x_pos,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_y_neg,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_y_pos,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_z_neg,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(bound_z_pos,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(init_pts,requires_grad=True,dtype=torch.float).to(device),
              torch.tensor(domain_pts,requires_grad=True,dtype=torch.float).to(device)])
    f.extend([['BC','-x'],['BC','+x'],['BC','-y'],['BC','+y'],['BC','-z'],['BC','+z'],['IC',T_ref],['domain']])
    
    return p,f

In [4]:
def BC(x,y,z,t,net,loc):
    X = torch.cat([x,y,z,t],axis=-1)
    T = net(X)
    if loc == '-x':
        T_x = grad(T,x,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return k*T_x - h*(T-T_ref) - Rboltz*emiss*(T**4-T_ref**4)
    if loc == '+x':
        T_x = grad(T,x,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return -k*T_x - h*(T-T_ref) - Rboltz*emiss*(T**4-T_ref**4)
    if loc == '-y':
        T_y = grad(T,y,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return k*T_y - h*(T-T_ref) - Rboltz*emiss*(T**4-T_ref**4)
    if loc == '+y':
        T_y = grad(T,y,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return -k*T_y - h*(T-T_ref) - Rboltz*emiss*(T**4-T_ref**4)
    if loc == '-z':
        T_t = grad(T,t,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        return T_t
    if loc == '+z':
        T_z = grad(T,z,create_graph=True,grad_outputs=torch.ones_like(T))[0]
        q = 2*P*eta/3.14159265/r**2*torch.exp(-2*(torch.square(x-x0-v*t)+torch.square(y-y0))/r**2)*(t<=t_end)*(t>0)
        return -k*T_z - h*(T-T_ref) - Rboltz*emiss*(T**4-T_ref**4) + q

In [5]:
def output_transform(X):
    X = T_range*nn.Softplus()(X)+ T_ref
    return X


def input_transform(X):
    X = 2.*(X-X_min)/(X_max-X_min) - 1.
    return X

In [3]:
import torch
import torch.nn as nn
from torch.autograd import grad
import numpy as np
import time
import matplotlib.pyplot as plt

# Define the plotting functions
def plot_points_2D(point_sets, flags, filename="2D_top_surface.png"):
    plt.figure(figsize=(10, 8))
    markers = {'domain': 'o', 'BC': 'x', 'IC': 's'}
    colors = {'domain': 'blue', 'BC': 'red', 'IC': 'green'}
    
    for points, flag in zip(point_sets, flags):
        points = points.detach().cpu().numpy()
        label = f"{flag[0]} {flag[1]}" if flag[0] == 'BC' else flag[0]
        plt.scatter(points[:, 0], points[:, 1], c=colors[flag[0]], marker=markers[flag[0]], label=label, s=1)

    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())

    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('2D Top Surface')
    plt.savefig(filename)
    plt.close()

def plot_points_3D(point_sets, flags, filename="3D_domain.png"):
    fig = plt.figure(figsize=(15, 12), dpi=300)
    ax = fig.add_subplot(111, projection='3d')
    markers = {'domain': 'o', 'BC': 'x', 'IC': 's'}
    colors = {
        'domain': 'blue', 
        'BC': 'red', 
        'IC': 'green', 
        'BC -x': 'cyan', 
        'BC +x': 'magenta', 
        'BC -y': 'yellow', 
        'BC +y': 'black', 
        'BC -z': 'purple', 
        'BC +z': 'orange'
    }
    
    for points, flag in zip(point_sets, flags):
        points = points.detach().cpu().numpy()
        label = f"{flag[0]} {flag[1]}" if flag[0] == 'BC' else flag[0]
        ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=colors.get(label, 'blue'), marker=markers[flag[0]], label=label, s=20, alpha=0.6)

    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='best', fontsize='small')

    ax.set_xlabel('x', fontsize=12)
    ax.set_ylabel('y', fontsize=12)
    ax.set_zlabel('z', fontsize=12)
    ax.set_title('3D Scatter Plot of Points in the Domain', fontsize=15)

    xlim = ax.get_xlim3d()
    ylim = ax.get_ylim3d()
    zlim = ax.get_zlim3d()
    ax.set_box_aspect([xlim[1] - xlim[0], ylim[1] - ylim[0], zlim[1] - zlim[0]])

    plt.savefig(filename, bbox_inches='tight', pad_inches=0.1)
    plt.close()

def plot_loss_curves(l_history, filename="loss_curves.png"):
    l_history = np.array(l_history)
    plt.figure(figsize=(10, 8))
    plt.plot(l_history[:, 0], label='Total Loss')
    plt.plot(l_history[:, 1], label='BC Loss')
    plt.plot(l_history[:, 2], label='IC Loss')
    plt.plot(l_history[:, 3], label='PDE Loss')
    plt.yscale('log')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Curves')
    plt.legend()
    plt.savefig(filename)
    plt.close()

def plot_predictions_3D(net, X_min, X_max, T_range, T_ref, epoch, filename="predictions_3D.png"):
    x = np.linspace(X_min.cpu().numpy()[0], X_max.cpu().numpy()[0], 50)
    y = np.linspace(X_min.cpu().numpy()[1], X_max.cpu().numpy()[1], 50)
    z = np.linspace(X_min.cpu().numpy()[2], X_max.cpu().numpy()[2], 50)
    t_vals = [0, 1, 2, 3]

    fig = plt.figure(figsize=(20, 15), dpi=300)

    for i, t in enumerate(t_vals):
        ax = fig.add_subplot(2, 2, i + 1, projection='3d')
        X, Y, Z = np.meshgrid(x, y, z)
        points = np.stack([X.ravel(), Y.ravel(), Z.ravel(), np.full_like(X.ravel(), t)], axis=-1)
        points = torch.tensor(points, dtype=torch.float32).to(next(net.parameters()).device)
        predictions = net(points).detach().cpu().numpy().reshape(50, 50, 50)

        sc = ax.scatter(X, Y, Z, c=predictions.ravel(), cmap='jet', marker='o', s=20, alpha=0.6)
        cbar = fig.colorbar(sc, ax=ax, label='U(t,x,y,z)')
        cbar.ax.tick_params(labelsize=12)
        ax.set_title(f"Time: {t}", fontsize=15)
        ax.set_xlabel('x', fontsize=12)
        ax.set_ylabel('y', fontsize=12)
        ax.set_zlabel('z', fontsize=12)

        xlim = ax.get_xlim3d()
        ylim = ax.get_ylim3d()
        zlim = ax.get_zlim3d()
        ax.set_box_aspect([xlim[1] - xlim[0], ylim[1] - ylim[0], zlim[1] - zlim[0]])

    plt.suptitle(f"3D Predictions at Epoch {epoch}", fontsize=18)
    plt.tight_layout()
    plt.savefig(filename, bbox_inches='tight', pad_inches=0.1)
    plt.close()

# PDE definition
def PDE(x, y, z, t, net):
    """
    Define the PDE for the problem.
    """
    X = torch.cat([x, y, z, t], axis=-1)
    T = net(X)

    T_t = grad(T, t, create_graph=True, grad_outputs=torch.ones_like(T))[0]
    T_x = grad(T, x, create_graph=True, grad_outputs=torch.ones_like(T))[0]
    T_xx = grad(T_x, x, create_graph=True, grad_outputs=torch.ones_like(T_x))[0]

    T_y = grad(T, y, create_graph=True, grad_outputs=torch.ones_like(T))[0]
    T_yy = grad(T_y, y, create_graph=True, grad_outputs=torch.ones_like(T_y))[0]

    T_z = grad(T, z, create_graph=True, grad_outputs=torch.ones_like(T))[0]
    T_zz = grad(T_z, z, create_graph=True, grad_outputs=torch.ones_like(T_z))[0]

    f = rho * Cp * T_t - k * (T_xx + T_yy + T_zz)
    return f

# Generate points for training
def generate_points(p=[], f=[]):
    """
    Generate points for the domain and boundary conditions.
    """
    t = np.linspace(x_min[3] + 0.01, x_max[3], 61)

    # Boundary points
    bound_x_neg, _ = sampling_uniform(1., x_min, x_max, '-x', t)
    bound_x_pos, _ = sampling_uniform(1., x_min, x_max, '+x', t)
    bound_y_neg, _ = sampling_uniform(1., x_min, x_max, '-y', t)
    bound_y_pos, _ = sampling_uniform(1., x_min, x_max, '+y', t)
    bound_z_neg, _ = sampling_uniform(1., x_min, x_max, '-z', t)
    bound_z_pos, _ = sampling_uniform(1., x_min, x_max, '+z', t)

    bound_z_pos_more = []
    for ti in t:
        if ti <= t_end:
            zi, _ = sampling_uniform(.25,
                                     [max(x0 + ti * v - 2 * r, x_min[0]), max(x_min[1], y0 - 2 * r), x_min[2]],
                                     [min(x0 + ti * v + 2 * r, x_max[0]), min(x_max[1], y0 + 2 * r), x_max[2]],
                                     '+z', [ti])
            bound_z_pos_more.append(zi)

    bound_z_pos_more = np.vstack(bound_z_pos_more)
    bound_z_pos = np.vstack((bound_z_pos, bound_z_pos_more))

    # Domain points
    domain_pts1, _ = sampling_uniform(2., [x_min[0], x_min[1], x_min[2]], [x_max[0], x_max[1], x_max[2] - 3.], 'domain', t)
    domain_pts2, _ = sampling_uniform(1., [x_min[0], x_min[1], x_max[2] - 3. + .5], [x_max[0], x_max[1], x_max[2] - 1.], 'domain', t)

    domain_pts3 = []
    for ti in t:
        di, _ = sampling_uniform(.5, [x_min[0], x_min[1], x_max[2] - 1. + .25], [x_max[0], x_max[1], x_max[2]], 'domain', [ti])
        domain_pts3.append(di)
    domain_pts3 = np.vstack(domain_pts3)
    domain_pts = np.vstack((domain_pts1, domain_pts2, domain_pts3))

    # Initial points
    init_pts1, _ = sampling_uniform(2., [x_min[0], x_min[1], x_min[2]], [x_max[0], x_max[1], x_max[2]], 'domain', [0], e=0)
    init_pts2, _ = sampling_uniform(.5, [x0 - 2, y0 - 2, x_max[2] - 2], [x0 + 2, y0 + 2, x_max[2]], 'domain', [0])

    init_pts = np.vstack((init_pts1, init_pts2))

    p.extend([torch.tensor(bound_x_neg, requires_grad=True, dtype=torch.float).to(device),
              torch.tensor(bound_x_pos, requires_grad=True, dtype=torch.float).to(device),
              torch.tensor(bound_y_neg, requires_grad=True, dtype=torch.float).to(device),
              torch.tensor(bound_y_pos, requires_grad=True, dtype=torch.float).to(device),
              torch.tensor(bound_z_neg, requires_grad=True, dtype=torch.float).to(device),
              torch.tensor(bound_z_pos, requires_grad=True, dtype=torch.float).to(device),
              torch.tensor(init_pts, requires_grad=True, dtype=torch.float).to(device),
              torch.tensor(domain_pts, requires_grad=True, dtype=torch.float).to(device)])
    f.extend([['BC', '-x'], ['BC', '+x'], ['BC', '-y'], ['BC', '+y'], ['BC', '-z'], ['BC', '+z'], ['IC', T_ref], ['domain']])
    
    return p, f

# Boundary condition function
def BC(x, y, z, t, net, loc):
    """
    Define the boundary condition for the problem.
    """
    X = torch.cat([x, y, z, t], axis=-1)
    T = net(X)
    if loc == '-x':
        T_x = grad(T, x, create_graph=True, grad_outputs=torch.ones_like(T))[0]
        return k * T_x - h * (T - T_ref) - Rboltz * emiss * (T**4 - T_ref**4)
    elif loc == '+x':
        T_x = grad(T, x, create_graph=True, grad_outputs=torch.ones_like(T))[0]
        return -k * T_x - h * (T - T_ref) - Rboltz * emiss * (T**4 - T_ref**4)
    elif loc == '-y':
        T_y = grad(T, y, create_graph=True, grad_outputs=torch.ones_like(T))[0]
        return k * T_y - h * (T - T_ref) - Rboltz * emiss * (T**4 - T_ref**4)
    elif loc == '+y':
        T_y = grad(T, y, create_graph=True, grad_outputs=torch.ones_like(T))[0]
        return -k * T_y - h * (T - T_ref) - Rboltz * emiss * (T**4 - T_ref**4)
    elif loc == '-z':
        T_t = grad(T, t, create_graph=True, grad_outputs=torch.ones_like(T))[0]
        return T_t
    elif loc == '+z':
        T_z = grad(T, z, create_graph=True, grad_outputs=torch.ones_like(T))[0]
        q = 2 * P * eta / np.pi / r**2 * torch.exp(-2 * (torch.square(x - x0 - v * t) + torch.square(y - y0)) / r**2) * (t <= t_end) * (t > 0)
        return -k * T_z - h * (T - T_ref) - Rboltz * emiss * (T**4 - T_ref**4) + q

# Output transformation function
def output_transform(X):
    return T_range * nn.Softplus()(X) + T_ref

# Input transformation function
def input_transform(X):
    return 2. * (X - X_min) / (X_max - X_min) - 1.

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Domain
x_max = np.array([40., 10., 6., 3.])
x_min = np.array([0., 0., 0., 0.])
X_max = torch.tensor(x_max, dtype=torch.float).to(device)
X_min = torch.tensor(x_min, dtype=torch.float).to(device)

# Laser parameters
x0 = 5.
y0 = 5.
r = 1.5
v = 10. 
t_end = 3.
P = 500.
eta = .4

# Ambient temperature and max temperature range
T_ref = 298.
T_range = 3000.

# Material parameters
Cp = .5
k = .01
h = 2e-5
Rboltz = 5.6704e-14
emiss = .3
rho = 8e-3

# Network definition
net = FNN([4, 100, 100, 100, 1], nn.Tanh(), in_tf=input_transform, out_tf=output_transform)
net.to(device)

# Generate points and flags
point_sets, flags = generate_points([], [])

# Plot initial points
plot_points_2D(point_sets, flags, filename="/home/am23mtech11006/AM23MTECH11006/Physics informed_laser/2d_3d_domain_plotting/2D_top_surface_initial.png")
plot_points_3D(point_sets, flags, filename="/home/am23mtech11006/AM23MTECH11006/Physics informed_laser/2d_3d_domain_plotting/3D_domain_initial.png")

# Define optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=2e-4)

# Training parameters
iterations = 50000
info_num = 100
w = [2., 1., 1., 1.]  # Define the weights

n_bc = sum([points.shape[0] for points, flag in zip(point_sets, flags) if flag[0] == 'BC'])
n_ic = sum([points.shape[0] for points, flag in zip(point_sets, flags) if flag[0] == 'IC'])
n_PDE = sum([points.shape[0] for points, flag in zip(point_sets, flags) if flag[0] == 'domain'])

# Train the network
l_history = []
err_history = []
for epoch in range(iterations):
    net.train()
    optimizer.zero_grad()
    l_BC = l_IC = l_PDE = 0

    for points, flag in zip(point_sets, flags):
        if flag[0] == 'BC':
            f = BC(points[:, 0:1], points[:, 1:2], points[:, 2:3], points[:, 3:4], net, flag[1])
            l_BC += loss(f) * points.shape[0] / n_bc
        elif flag[0] == 'IC':
            pred = net(points)
            l_IC += loss(pred, flag[1]) * points.shape[0] / n_ic
        elif flag[0] == 'domain':
            f = PDE(points[:, 0:1], points[:, 1:2], points[:, 2:3], points[:, 3:4], net)
            l_PDE += loss(f) * points.shape[0] / n_PDE

    cost = (w[0] * l_BC + w[1] * l_IC + w[2] * l_PDE) / 3  # weighted
    l_history.append([cost.item(), l_BC.item(), l_IC.item(), l_PDE.item()])

    cost.backward()
    optimizer.step()

    # Print all types of losses in the console after every epoch
    # Print all types of losses in the console every 100 epochs
    if epoch % 100 == 0:
        print(f"Epoch {epoch}: Total Loss = {cost.item():.4e}, BC Loss = {l_BC.item():.4e}, IC Loss = {l_IC.item():.4e}, PDE Loss = {l_PDE.item():.4e}")

    # Save the model, loss curves, and plot predictions every 1000 epochs
    if epoch % 1000 == 0:
        torch.save(net.state_dict(), f'/home/am23mtech11006/AM23MTECH11006/Physics informed_laser/models/model_epoch_{epoch}.pt')
        np.save(f'/home/am23mtech11006/AM23MTECH11006/Physics informed_laser/loss_history_epoch_{epoch}.npy', l_history)
        plot_loss_curves(l_history, filename=f"/home/am23mtech11006/AM23MTECH11006/Physics informed_laser/loss_history/loss_curves_epoch_{epoch}.png")
        plot_predictions_3D(net, X_min, X_max, T_range, T_ref, epoch, filename=f"/home/am23mtech11006/AM23MTECH11006/Physics informed_laser/results_prediction_3d_plots/predictions_3D_epoch_{epoch}.png")

# Save the final trained model and loss history
torch.save(net.state_dict(), '/home/am23mtech11006/AM23MTECH11006/Physics informed_laser/models/no_auxiliary_data.pt')
np.save('/home/am23mtech11006/AM23MTECH11006/Physics informed_laser/models/no_auxiliary_data_loss.npy', l_history)
plot_loss_curves(l_history, filename="/home/am23mtech11006/AM23MTECH11006/Physics informed_laser/loss_history/loss_curves_final.png")


Epoch 0: Total Loss = 2.0586e+06, BC Loss = 6.3203e+03, IC Loss = 6.1632e+06, PDE Loss = 5.8997e-01
Epoch 100: Total Loss = 1.8564e+05, BC Loss = 1.3398e+05, IC Loss = 2.8894e+05, PDE Loss = 2.4728e+01
Epoch 200: Total Loss = 6.3677e+04, BC Loss = 4.5901e+04, IC Loss = 9.9200e+04, PDE Loss = 2.9691e+01
Epoch 300: Total Loss = 1.7743e+04, BC Loss = 1.2530e+04, IC Loss = 2.8140e+04, PDE Loss = 3.1001e+01
Epoch 400: Total Loss = 6.7726e+03, BC Loss = 4.7777e+03, IC Loss = 1.0733e+04, PDE Loss = 2.9665e+01
Epoch 500: Total Loss = 3.4394e+03, BC Loss = 2.4470e+03, IC Loss = 5.3967e+03, PDE Loss = 2.7260e+01
Epoch 600: Total Loss = 2.0724e+03, BC Loss = 1.4933e+03, IC Loss = 3.2062e+03, PDE Loss = 2.4561e+01
Epoch 700: Total Loss = 1.3884e+03, BC Loss = 1.0149e+03, IC Loss = 2.1133e+03, PDE Loss = 2.1884e+01
Epoch 800: Total Loss = 9.9804e+02, BC Loss = 7.4084e+02, IC Loss = 1.4931e+03, PDE Loss = 1.9363e+01
Epoch 900: Total Loss = 7.5412e+02, BC Loss = 5.6861e+02, IC Loss = 1.1081e+03, PDE 

: 